# MinimalOMEROImageJ
Using Python to apply ImageJ macros to images on OMERO.\
For the examples to work your default group must be Royle_Cooper and you must be connected to the vpn.

## Import packages

In [ ]:
import imagej
from omero.gateway import BlitzGateway
import getpass

## Define Functions

### Start Fiji
Change path to local version of Fiji.app folder

In [ ]:
def start_fiji():
    ij = imagej.init('/path/to/Fiji.app', headless=False)
    ij.getVersion()
    return ij

### Connecting and disconnecting from OMERO

In [ ]:
def connect(hostname, username, password):
    """
    Connect to an OMERO server
    :param hostname: Host name
    :param username: User
    :param password: Password
    :return: Connected BlitzGateway
    """
    conn = BlitzGateway(username, password,
                        host=hostname, secure=True)
    conn.connect()
    conn.c.enableKeepAlive(60)
    return conn


def disconnect(conn):
    """
    Disconnect from an OMERO server
    :param conn: The BlitzGateway
    """
    conn.close()

### Loading plane of image from OMERO
Plane is loaded as a numpy array

In [ ]:
def load_plane(image):
    """
    Load a 2D-plane as a numpy array
    :param image: The image
    """
    pixels = image.getPrimaryPixels()
    return pixels.getPlane(0, 0, 0)

### Apply ImageJ macro

In [ ]:
def analyse(ij, conn, image_id,macroPath):
    # Step 3 - Load image
    image = conn.getObject("Image", image_id)
    # -
    plane = load_plane(image)
    from jnius import autoclass
    autoclass('ij.WindowManager')
    ij.ui().show('Image', ij.py.to_java(plane))
    macro="""runMacro(""" + macroPath + """);"""
    ij.py.run_macro(macro)

## Method

In [ ]:
print("initializing fiji, this can take a bit of time...")
ij = start_fiji()

### Enter details and connect to OMERO

In [ ]:
username = input('Username: ') #Request user to enter OMERO username
password = getpass.getpass(prompt='Password: ') #Request user to enter OMERO password
conn=connect('camdu.warwick.ac.uk',username, password) #Use details to connect to OMERO server

## Enter OMERO image ID

In [ ]:
image_id = int(input("Image ID [246165]: ") or  246165)

In [ ]:
macroPath=input("Macro [ExampleMacro.ijm]:") or "ExampleMacro.ijm"
macroPath='"'+macroPath+'"'

In [ ]:
print("running the macro...")
analyse(ij, conn, image_id,macroPath)

In [ ]:
disconnect(conn)

## Troubleshooting

### Check connection to OMERO
Output should be: <omero.gateway._BlitzGateway object at NxNNNNNNNNNNNN> \
where N are numbers or letters

In [ ]:
print(conn)

### If image is NoneType check your current group. If wrong, you need to change this in omero.web
Output should list all groups you are a member of and highlight your current group at the end.\
If the current group is wrong you will need to change this in OMERO.web. \
Go to your name (top right corner) > User settings. Change the default group and hit save.

In [ ]:
print("Member of:")
for g in conn.getGroupsMemberOf():
    print("   ID:", g.getName(), " Name:", g.getId())
group = conn.getGroupFromContext()
print("Current group: ", group.getName())